<a href="https://colab.research.google.com/github/UrielMendoza/LANOT_USAC_taller_sentinel2/blob/main/ejemplo_API_rest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notebook 1: Descarga de Imágenes Sentinel-2 Nivel L2A**

Autor: urielm

Fecha: 20 de noviembre de 2024

Introducción

Este notebook muestra cómo buscar y descargar imágenes Sentinel-2 Nivel L2A para tiles específicos y fechas definidas utilizando la API de Copernicus Data Space.

1. Importar bibliotecas necesarias

In [1]:
import datetime
import os
import requests
import pandas as pd
import time


2. Configuración de credenciales y tiles
python
Copiar código


In [2]:
# Credenciales para Copernicus Data Space
Dataspace_username = ""  # Reemplaza con tu usuario
Dataspace_password = ""  # Reemplaza con tu contraseña

# Tiles de interés
tiles = {
    "prueba": ["16PCC"]  # Agrega aquí los tiles que necesitas
}


3. Obtener el token de acceso

In [3]:
def get_access_token(username: str, password: str) -> str:
    """
    Obtiene un token de acceso para la API de Copernicus DataSpace.
    """
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        response = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        response.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Error al obtener el token de acceso. Respuesta del servidor: {response.json()}"
        )
    return response.json()["access_token"]


4. Buscar productos Sentinel-2


In [4]:
def search_products(tile=None, date=None, product_type="L2A") -> dict:
    """
    Busca productos Sentinel-2 en la API de Copernicus DataSpace para un tile y fecha.
    """
    date_str = date.strftime("%Y%m%d")
    level = "MSIL2A" if product_type == "L2A" else "MSIL1C"
    query = f"{level}_{date_str}"

    url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name, '{query}') and contains(Name, '{tile}')"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception("Error al buscar productos.")
    json_response = response.json()

    if not json_response["value"]:
        raise Exception(f"No se encontraron productos para el tile {tile} y fecha {date_str}.")

    product_info = json_response["value"][0]
    return {
        'id': product_info['Id'],
        'name': product_info['Name'],
        'tile': tile,
        'origin_date': product_info['OriginDate'],
    }


5. Descargar productos

In [5]:
def download_products(product: dict, datadir: str, max_retries=3):
    """
    Descarga un producto Sentinel-2 de la API de Copernicus DataSpace.
    """
    access_token = get_access_token(Dataspace_username, Dataspace_password)
    headers = {"Authorization": f"Bearer {access_token}"}

    product_id = product['id']
    product_name = product['name']
    tile = product['tile']
    url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"

    print(f"Intentando descargar desde: {url}")

    for attempt in range(max_retries):
        response = requests.get(url, headers=headers, stream=True)
        if response.status_code == 200:
            break
        print(f"Intento {attempt + 1} fallido: {response.status_code}")
        print(f"Detalle del error: {response.text}")  # Depurar errores
        time.sleep(5)

    if response.status_code != 200:
        raise Exception(f"Error al descargar el producto después de {max_retries} intentos. Código: {response.status_code}, Detalle: {response.text}")

    tile_dir = os.path.join(datadir, f"T{tile}")
    os.makedirs(tile_dir, exist_ok=True)

    file_path = os.path.join(tile_dir, f"{product_name}.zip")
    with open(file_path, 'wb') as file:
        print(f'Descargando {product_name}...')
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                file.write(chunk)
    print(f"Descarga completa: {file_path}")


6. Buscar y descargar datasets

In [6]:
def search_and_download_datasets(tiles, start_date, datadir):
    """
    Busca y descarga productos Sentinel-2 para una lista de tiles y una fecha específica.
    """
    for tile in tiles:
        try:
            product = search_products(tile, start_date, product_type="L2A")
            print(f"Producto encontrado: {product}")
            download_products(product, datadir)
        except Exception as e:
            print(f"Error al procesar el tile {tile}: {e}")


7. Ejecutar el proceso

In [7]:
if __name__ == "__main__":
    # Configuración de directorio de salida
    datadir = './datos/'  # Directorio donde se guardarán los productos

    # Fecha de interés
    start_date = datetime.date(2024, 5, 5)

    print(f"Iniciando búsqueda y descarga para la fecha: {start_date}")
    search_and_download_datasets(tiles["prueba"], start_date, datadir)


Iniciando búsqueda y descarga para la fecha: 2024-05-05
Producto encontrado: {'id': 'ebd6ec04-afc7-4340-ac3c-ba0c686b0b67', 'name': 'S2A_MSIL2A_20240505T160901_N0510_R140_T16PCC_20240505T235451.SAFE', 'tile': '16PCC', 'origin_date': '2024-05-06T00:50:53.000000Z'}
Intentando descargar desde: https://zipper.dataspace.copernicus.eu/odata/v1/Products(ebd6ec04-afc7-4340-ac3c-ba0c686b0b67)/$value
Descargando S2A_MSIL2A_20240505T160901_N0510_R140_T16PCC_20240505T235451.SAFE...
Descarga completa: ./datos/T16PCC/S2A_MSIL2A_20240505T160901_N0510_R140_T16PCC_20240505T235451.SAFE.zip
